# SCP Toolbox Workshop

___A tutorial on generating dynamically feasible trajectories reliably and efficiently___

Monday, February 7, 2022

Rocky Mountain AAS GN&C Conference, Breckenridge, CO

In [1]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

using SCPToolbox
using PyPlot, Colors, LinearAlgebra

# Import the different possible low-level convex solvers
using COSMO
using Ipopt
using SCS
using ECOS
using OSQP

  Activating project at `~/Nextcloud/personal/research/scp/aas_gnc_workshop/tutorial`
   Resolving package versions...
  No Changes to `~/Nextcloud/personal/research/scp/aas_gnc_workshop/tutorial/Project.toml`
  No Changes to `~/Nextcloud/personal/research/scp/aas_gnc_workshop/tutorial/Manifest.toml`
┌ Info: Precompiling SCPToolbox [bd2bc758-a5dc-11eb-050f-6b2434548817]
└ @ Base loading.jl:1423


# Part 4: Rocket-Landing Guidance

In this part of the tutorial, you will participate in formulating and solving a Lunar rocket landing trajectory optimization problem.

The rocket has a 2-dimensional position and a rotation, and a gimbaled rocket engine. The goal is to take the rocket from an airborne state to a soft touchdown on the ground.

<center>
    <img src="media/p4-rocket-landing-overview.png"
         alt="Dubin's car overview"
         style="background-color: white; width: 300px; display: block; margin-left: auto; margin-right: auto;"/>
    <br />
    <b>Figure.</b> Illustration of the rocket soft landing problem.
    <br />
    <br />
</center>

Begin by defining a new `TrajectoryProblem` object.

In [ ]:
pbm = # Enter your code here

<details>

<summary>Answer</summary>



```julia
pbm = TrajectoryProblem();
```
</details>

## Rocket parameters

Any trajectory optimization problem begins by defining the constant parameters of the problem. For example, how much does the rocket weigh? How high up does it start?

We define these below.

In [27]:
# Environment parameters
g = 1.625 # [m/s^2] Gravitational acceleration of the planet
g_E = 9.807; # [m/s^2] Earth gravity

In [29]:
# Mechanical parameters
m_wet = 25e3 # [kg] Initial mass
L = 5.0 # [m] Thrust lever arm
J = 100e3; # [kg*m^2] Moment of inertia

In [31]:
# Propulsion parameters
Isp = 370 # [s] Specific impulse
T_min = 20e3 # [N] Minimum thrust
T_max = 80e3 # [N] Maximum thrust
α = 1 / (Isp * g_E) # [kg/s/N] Fuel consumption rate coefficient
δ_max = deg2rad(5.0); # [rad] Maximum gimbal angle

In [32]:
# Trajectory parameters
γ_gs = deg2rad(45.0); # [rad] Glideslope angle (measured from the horizon)

## Rocket dynamics

The partial rocket dynamics are given by the following ordinary differential equations:

\begin{align}
\dot x &= v_x, \\
\dot y &= \mathrm{\color{grey}{TODO}}, \\
\dot v_x &= -\frac{T}{m}\sin(\theta+\delta), \\
\dot v_y &= \mathrm{\color{grey}{TODO}}, \\
\dot \theta &= \mathrm{\color{grey}{TODO}}, \\
\dot \omega &= -\frac{LT}{J}\sin(\delta), \\
\dot m &= -\alpha T.
\end{align}

<details>

<summary>Answer</summary>

\begin{align}
    \dot y &= v_y, \\
    \dot v_y &= \frac{T}{m}\cos(\theta+\delta)-g, \\
    \dot \theta &= \omega.
\end{align}

</details>

Define the state vector $x\in\mathbb R^n$, with the elements in the same order as they appear on the left-hand side of the ordinary differential equations above:

$$
x = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
x = \begin{bmatrix}
x \\ y \\ v_x \\ v_y \\ \theta \\ \omega \\ m
\end{bmatrix}
$$

</details>

Define the input vector $u\in\mathbb R^m$, with the thrust coming first:

$$
u = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
u = \begin{bmatrix}
T \\ \delta
\end{bmatrix}
$$

</details>

Complete the function below that evaluates the right-hand side of the dynamics ordinary differential equations above. The function signature is:

$$
\dot x = f_{rocket}(x, u),
$$

where the arguments are the state and input and the output is the state time derivative.

In [3270]:
f_rocket(x, u) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    # Enter your code here
end;

<details>

<summary>Answer</summary>

```julia
f_rocket(x, u) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    return [vx; vy; -(T/m)*sin(θ+δ); (T/m)*cos(θ+δ)-g; ω; -(L*T/J)*sin(δ); -α*T]*tf
end;
```

</details>

Remember from the Dubin's car example that SCP Toolbox operates on a normalized-time problem where time takes values in $[0,1]$.

We are going to solve a free-final time rocket landing problem, and this requires us to introduce a one-element parameter vector that holds the time dilation coefficient:

$$
p = \begin{bmatrix}
t_f
\end{bmatrix}.
$$

The normalized-time dynamics are given by:

$$
\dot x = t_f f_{rocket}(x, u) \equiv f(x, u, p).
$$

This allows us to write the normalized-time dynamics in Julia as follows.

In [3271]:
f(x, u, p) = begin
    tf, = p
    f_rocket(x, u)*tf
end;

Based on the above definitions of the state ($x$), the input ($u$), and the parameter ($p$), record their dimensions ($n$, $m$, and $d$, respectively).

In [ ]:
n = # Enter your code here
m = # Enter your code here
d = # Enter your code here

<details>

<summary>Answer</summary>

```julia
n = 7
m = 2
d = 1
```

</details>

Use the SCP Toolbox API to define the state, input, and parameter dimensions in the problem.

In [5]:
# Enter your code here

<details>

<summary>Answer</summary>

```julia
problem_set_dims!(pbm, n, m, d)
```

</details>

### Jacobians

We need to define the Jacobians of the dynamics with respect to the state, input, and parameter vectors.

Remember the definition of the Jacobian of a general function $g(z):\mathbb R^n\to\mathbb R^m$:

$$
\nabla_z g(z) = \begin{bmatrix}
\frac{\partial g(z)}{\partial z_1} &
\frac{\partial g(z)}{\partial z_2} &
\cdots &
\frac{\partial g(z)}{\partial z_n}
\end{bmatrix}\in\mathbb R^{m\times n}.
$$

With the above in mind, write down "in math" the Jacobian of the dynamics with respect to $x$:

$$
A(x, u, p) = \nabla_x f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times n}.
$$

To help, here is what the column corresponding to the state $x_5\equiv\theta$ looks like:

$$
\begin{bmatrix}
0\\0\\- \frac{T t_f \cos{\left(\delta + \theta \right)}}{m}\\- \frac{T t_f \sin{\left(\delta + \theta \right)}}{m}\\0\\0\\0\end{bmatrix}.
$$

<details>

<summary>Answer</summary>

$$
A(x, u, p) = 
\begin{bmatrix}
0 & 0 & t_f & 0 & 0 & 0 & 0\\
0 & 0 & 0 & t_f & 0 & 0 & 0\\
0 & 0 & 0 & 0 & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m} & 0 & \frac{T t_f \sin{\left(\delta + \theta \right)}}{m^{2}}\\0 & 0 & 0 & 0 & - \frac{T t_f \sin{\left(\delta + \theta \right)}}{m} & 0 & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m^{2}}\\0 & 0 & 0 & 0 & 0 & t_f & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}.
$$

</details>

Now implement the function in Julia.

In [7]:
A(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
end;

<details>

<summary>Answer</summary>

```julia
A(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [0 0 1 0       0       0       0
            0 0 0 1       0       0       0
            0 0 0 0 -T/m*cos(θ+δ) 0 T/m^2*sin(θ+δ)
            0 0 0 0 -T/m*sin(θ+δ) 0 -T/m^2*cos(θ+δ)
            0 0 0 0       0       1       0
            0 0 0 0       0       0       0
            0 0 0 0       0       0       0]*tf
end
```

</details>

Similarly, write down the Jacobian with respect to the input, $u$:

$$
B(x, u, p) = \nabla_u f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times m}.
$$

<details>

<summary>Answer</summary>

$$
B(x, u, p) = 
\begin{bmatrix}
0 & 0\\0 & 0\\- \frac{t_f \sin{\left(\delta + \theta \right)}}{m} & - \frac{T t_f \cos{\left(\delta + \theta \right)}}{m}\\\frac{t_f \cos{\left(\delta + \theta \right)}}{m} & - \frac{T t_f \sin{\left(\delta + \theta \right)}}{m}\\0 & 0\\- \frac{L t_f \sin{\left(\delta \right)}}{J} & - \frac{L T t_f \cos{\left(\delta \right)}}{J}\\- \alpha t_f & 0
\end{bmatrix}.
$$

</details>

Now implement the function in Julia.

In [9]:
B(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
end;

<details>

<summary>Answer</summary>

```julia
B(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return [     0            0
                 0            0
            -sin(θ+δ)/m -T/m*cos(θ+δ)
             cos(θ+δ)/m -T/m*sin(θ+δ)
                 0            0
            -L/J*sin(δ) -L*T/J*cos(δ)
                -α            0       ]*tf
end
```

</details>

Finally, write down the Jacobian with respect to the parameter vector, $p$:

$$
F(x, u, p) = \nabla_p f(x, u, p) = \begin{bmatrix}
\mathrm{\color{grey}{TODO}}
\end{bmatrix}\in\mathbb R^{n\times d}.
$$

<details>

<summary>Answer</summary>

$$
F(x, u, p) = 
f_{rocket}(x, u).
$$

</details>

Now implement the function in Julia.

In [10]:
F(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    # Enter your code here
end;

<details>

<summary>Answer</summary>

```julia
F(x, u, p) = begin
    rx, ry, vx, vy, θ, ω, m = x
    T, δ = u
    tf, = p
    return reshape(f(t, x, u, p)/tf, 7, 1)
end;
```

</details>

The SCP Toolbox API also passes other arguments to the dynamics and its Jacobians:
* `t`: the time;
* `k`: the discrete time index;
* `pbm`: the `TrajectoryProblem` object that we defined at the very beginning.

We write a simple wrapper to satisfy the API.

In [ ]:
wrap(func) = (t, k, x, u, p, pbm) -> func(t, x, u, p);
f_scp = wrap(f)
A_scp = wrap(A)
B_scp = wrap(B)
F_scp = wrap(F)

Use the above functions `f_scp`, `A_scp`, `B_scp`, `F_scp` in order to define the dynamics using the SCP Toolbox API.

In [24]:
# Enter your code here

<details>

<summary>Answer</summary>

```julia
problem_set_dynamics!(pbm, wrap(f), wrap(A), wrap(B), wrap(F))
```

</details>

## Boundary Conditions

Let's define the boundary conditions on the state as a fixed "airborne" point above the landing site at the start, and a point at the landing state at the end of the trajectory. Because the rocket depletes mass, and we do not know by how much ahead of time, the final mass is left unspecified.

In [35]:
km2m(x) = x*1e3 # Conversion from km to m
kph2mps(v) = v/3.6 # Conversion from km/h to m/s

# Initial state
x_0 = [km2m(0.5)
       km2m(1.4)
       kph2mps(80)
       kph2mps(-100)
       deg2rad(-30.0)
       deg2rad(0.0)
       m_wet]

# Terminal state (without mass)
x_f = zeros(6);

Recall that the boundary conditions are defined as the equality constraints:

\begin{align}
g_{ic}(x(0), p) &= 0, \\
g_{tc}(x(1), p) &= 0.
\end{align}

Given the definitions of $x_0$ (`x_0`) and $x_f$ (`x_f`), define the $g_{ic}$ and $g_{tc}$ functions.

\begin{align}
g_{ic}(x, p) &= \mathrm{\color{grey}{TODO}}, \\
g_{tc}(x, p) &= \mathrm{\color{grey}{TODO}}.
\end{align}

<details>

<summary>Answer</summary>

\begin{align}
g_{ic}(x, p) &= x-x_0, \\
g_{tc}(x, p) &= E_{m} x-x_f,
\end{align}
where:
$$
E_{m} = \begin{bmatrix}
I\in\mathbb R^{6\times 6} &\mid & 0
\end{bmatrix}\in\mathbb R^{6\times 7}.
$$

</details>

Now implement these functions in Julia.

In [37]:
g_ic(x, p) = # Enter your code here
g_tc(x, p) = # Enter your code here

<details>

<summary>Answer</summary>

```julia
g_ic(x, p) = x-x_0
g_tc(x, p) = x[1:6]-x_f;
```

</details>

## State Constraints

In order to not collide with the ground or any obstacles (such as hills) surrounding the landing site, the rocket's position is constrained to be above a certain glideslope, as illustrated in the figure at the top of this notebook.

Write down the glideslope constraint on the position vector $[x;y]$. (Hint: it is a convex inequality and involves $|x|$).

$$
\mathrm{\color{grey}{TODO}}\le 0.
$$

<details>

<summary>Answer</summary>

$$
|x|-\frac{y}{\tan(\gamma_{gs})}\le 0.
$$

</details>

What kind of convex cone can represent this constraint?

Enter your answer in term of the Julia variable that is used to by the SCP Toolbox to denote the cone (recall the table from the Part 1 notebook): `TODO`.

<details>

<summary>Answer</summary>

`L1`

</details>

Now use the SCP Toolbox's `@add_constraint` macro (which you learned about in Part 1) in order to define the glideslope constraint. Assume that the function receives `x` (the full state vector) and `clp` (the `ConicProgram` object).

In [ ]:
glideslope(x, clp) = # Enter your code here

<details>

<summary>Answer</summary>

```julia
glideslope(x, clp) = @add_constraint(clp, L1, x -> begin
        x, y = x[1], x[2]
        return [y/tan(γ_gs); x]
    end);
```
    
</details>

Let's associate this constraint with the trajectory problem using the SCP Toolbox API for convex state constraints.

In [ ]:
problem_set_X!(pbm, (t, k, x, p, pbm, clp) -> glideslope(x, clp))

## Control Constraints

The control constraints ensure that the rocket's propulsion system can actually reproduce the inputs that are computed by the optimal trajectory.

For this problem, we will constraint the thrust to $T\in [T_\min, T_\max]$ and the gimbal angle $\delta\in [-\delta_\max,\delta_\max]$.

Write these control constraints as cones. Write the constraints separately for thrust lower bound, upper bound, and gimbal angle (thus you should end up with 3 cones). Only use the cones that are supported by the SCP Toolbox (refer to Part 1).

<details>

<summary>Answer</summary>

\begin{align*}
T_{\min}-T &\in \mathbb K_{\le 0}, \\
T-T_{\max} &\in \mathbb K_{\le 0}, \\
\begin{bmatrix}
\delta_{\max} \\ \delta
\end{bmatrix} &\in \mathbb K_{1}.
\end{align*}
    
</details>

Now implement each constraint in Julia using `@add_constraint`.

In [ ]:
thrust_upper_bound(T, clp) = # Enter your code here
thrust_lower_bound(T, clp) = # Enter your code here
gimbal_limit(δ, clp) = # Enter your code here

<details>

<summary>Answer</summary>

```julia
thrust_upper_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T-T_max)
thrust_lower_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T_min-T)
gimbal_limit(δ, clp) = @add_constraint(clp, L1, δ -> vcat(δ_max, δ));
```
    
</details>

Now call the SCP Toolbox API function `problem_set_U!` in order to associate the convex input constraints with the trajectory problem.

In [ ]:
problem_set_U!(pbm, (t, k, u, p, pbm, clp) -> begin
        # Enter your code here
    end)

<details>

<summary>Answer</summary>

```julia
problem_set_U!(pbm, (t, k, u, p, pbm, clp) -> begin
        T, δ = u
        thrust_upper_bound(T, clp)
        thrust_lower_bound(T, clp)
        gimbal_limit(δ, clp)
    end)
```
    
</details>

## Objective Function

## Initial Trajectory Guess

## Variable Scaling

## Solve the Problem Using PTR

In [25]:
# Parameters
N, Nsub = 20, 20
iter_max = 30
disc_method = FOH
wvc, wtr = 1e1, 1e-1
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-4
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>0)      

pars = PTR.Parameters(N, Nsub, iter_max, disc_method, wvc, wtr, ε_abs,
                      ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

## Plots

-----------------

In [ ]:
wrap(func) = (t, k, x, u, p, pbm) -> func(t, x, u, p)
problem_set_dynamics!(pbm, wrap(f), wrap(A), wrap(B), wrap(F))

## Boundary Conditions

The initial and terminal boundary conditions on the trajectory are set by the following two constraints:

\begin{align}
g_{ic}(x(0), p) &= 0, \\
g_{tc}(x(1), p) &= 0.
\end{align}

In [ ]:
km2m(x) = x*1e3 # Conversion from km to m
kph2mps(v) = v/3.6 # Conversion from km/h to m/s

x_0 = [km2m(0.5); km2m(1.4); kph2mps(80); kph2mps(-100); deg2rad(-30.0); deg2rad(0.0); m_wet]
x_f = zeros(6)

g_ic(x, p) = x-x_0
g_tc(x, p) = x[1:6]-x_f;

Again, we need to provide the Jacobians of $g_{ic}$ and $g_{tc}$, since in general these may be nonaffine functions.

\begin{align}
H_0(x(0), p) &= \nabla_x g_{ic}(x(0), p), \\
K_0(x(0), p) &= \nabla_p g_{ic}(x(0), p), \\
H_f(x(1), p) &= \nabla_x g_{tc}(x(1), p), \\
K_f(x(1), p) &= \nabla_p g_{tc}(x(1), p).
\end{align}

When a Jacobian is not provided, the SCP Toolbox assumes that it is zero.

In [ ]:
H_0(x, p) = I(7)
H_f(x, p) = collect(hcat(I(6), zeros(6)));

In [ ]:
wrap(func) = (x, p, pbm) -> func(x, p)
problem_set_bc!(pbm, :ic, wrap(g_ic), wrap(H_0))
problem_set_bc!(pbm, :tc, wrap(g_tc), wrap(H_f))

## State constraints

In [ ]:
tf_min = 40.0 # [s] Minimum time of flight
nonnegative_time_of_flight(tf, clp) = @add_constraint(clp, NONPOS, tf -> tf_min-tf);

In [ ]:
glide_slope(x, clp) = @add_constraint(clp, L1, x -> begin
        x, y = x[1], x[2]
        return [y/tan(γ_gs); x]
    end);

In [ ]:
problem_set_X!(pbm, (t, k, x, p, pbm, clp) -> begin
        tf = p[1]
        nonnegative_time_of_flight(tf, clp)
        glide_slope(x, clp)
    end)

## Control constraints

In [ ]:
thrust_upper_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T-T_max)
thrust_lower_bound(T, clp) = @add_constraint(clp, NONPOS, T -> T_min-T)
gimbal_limit(δ, clp) = @add_constraint(clp, L1, δ -> vcat(δ_max, δ));

In [ ]:
problem_set_U!(pbm, (t, k, u, p, pbm, clp) -> begin
        T, δ = u[1], u[2]
        thrust_upper_bound(T, clp)
        thrust_lower_bound(T, clp)
        gimbal_limit(δ, clp)
    end)

## Objective Function

1000.0Borrowing from the standard choice in optimal control literature, the SCP Toolbox works with an objective function in the Bolza form:

$$
J(x, u, p) = \phi(x(1), p) + \int_0^1 \Gamma(x(t), u(t), p) dt.
$$

The function $\phi(\cdot)\in\mathbb R$ defines the terminal cost, while $\Gamma(\cdot)\in\mathbb R$ defines the running cost. Both have to be convex, at most quadratic, functions.

> ℹ️ Any nonconvexity in the cost can in general be off-loaded into the constraints by defining additional state, control, and/or parameter variables.

In [ ]:
ϕ(x, p) = -x[end]/m_wet;

The SCP Toolbox provides the following API functions for defining the cost:
* `problem_set_terminal_cost!`: if not provided, it is assumed that $\phi\equiv 0$;
* `problem_set_running_cost!`: if not provided, it is assumed that $\Gamma\equiv 0$;

In [ ]:
wrap(func) = (x, p, pbm) -> func(x, p)
problem_set_terminal_cost!(pbm, wrap(ϕ))

In [ ]:
# Γ(x, u, p) = ( u[1]/T_max )^2
# wrap(func) = (t, k, x, u, p, pbm) -> func(x, u, p)
# problem_set_running_cost!(pbm, :ptr, wrap(Γ))

## Initial Trajectory Guess

SCP algorithms require an initial guess for the trajectory. This can be very simple, and does not need to be feasible.

For Dubin's car, we will use a straight line for the state and zero for the input.

Given a time $t\in [0,1]$, the state initial guess will be:

\begin{align}
\bar x(t) &= (1-t) x_0 + t x_f, \\
\bar u(t) &= 0.
\end{align}

SCP algorithms work by discretizing the trajectory problem into $N$ temporal nodes.

The API provides a function `problem_set_guess!` for defining the initial guess, discretized into $N$ points.

For convenience, a function `straightline_interpolate` is available to define a straight line guess.

In [ ]:
T_guess = T_min+0.5*(T_max-T_min) # [N] Initial thrust guess
v_0, v_f = x_0[3:4], x_f[3:4]
tf_guess = norm(v_f-v_0, 2)/(T_guess/m_wet-g)
Δm_guess = -α*T_guess*tf_guess
m_dry_guess = x_0[end]+Δm_guess

state_guess(N) = straightline_interpolate(x_0, vcat(x_f, m_dry_guess), N)
input_guess(N) = straightline_interpolate([T_guess; 0.0], [T_guess; 0.0], N);

In [ ]:
problem_set_guess!(pbm, (N, pbm) -> begin
    x = state_guess(N)
    u = input_guess(N)
    p = [tf_guess]
    return x, u, p
end)

## Variable Scaling

Variable scaling is important when the solution variables are of widely different magnitudes.

The SCP Toolbox performs automatic scaling for variables whose magnitudes are naturally constrained by the convex constraints.

However, you can also set the variable scaling manually by providing the expected value ranges for each variable.

This is done using the SCP Toolbox API function `problem_advise_scale!!`.

In [ ]:
rx_range_box = max(x_0[1], 1.0)
ry_range_box = max(x_0[2], 1.0)
vx_range_box = max(abs(x_0[3]), 1.0)
vy_range_box = min(x_0[4], -1.0)
θ_range_box = deg2rad(20.0)
ω_range_box = max(abs(x_0[6]), deg2rad(5.0))
m_range_box = m_wet

problem_advise_scale!(pbm, :state, 1, (-rx_range_box, rx_range_box))
problem_advise_scale!(pbm, :state, 2, (0, ry_range_box))
problem_advise_scale!(pbm, :state, 3, (-vx_range_box, vx_range_box))
problem_advise_scale!(pbm, :state, 4, (vy_range_box, 0.0))
problem_advise_scale!(pbm, :state, 5, (-θ_range_box, θ_range_box))
problem_advise_scale!(pbm, :state, 6, (-ω_range_box, ω_range_box))
problem_advise_scale!(pbm, :state, 7, (m_wet+Δm_guess, m_wet))

δ_range_box = deg2rad(1.0)

problem_advise_scale!(pbm, :input, 1, (T_min, T_max))
problem_advise_scale!(pbm, :input, 2, (-δ_range_box, δ_range_box))

problem_advise_scale!(pbm, :parameter, 1, (0.1*tf_guess, tf_guess))

## Configuring the SCP Solver

The SCP Toolbox provides several SCP algorithm choices: `PTR`, `SCvx`, and `GuSTO`.

Each algorithm must be provided with a set of parameters that define its behavior. Most of the parameters are shared, but some are algorithm-specific.

SCP algorithm performance depends significantly on a well-chosen set of parameters, and this is problem-dependent.

A lot of intuition is documented in our [Control Systems Magazine article](https://arxiv.org/abs/2106.09125).

In [ ]:
# Parameters
N, Nsub = 20, 20
iter_max = 30
disc_method = FOH
wvc, wtr = 1e1, 1e-1
feas_tol = 5e-3
ε_abs, ε_rel = 1e-5, 1e-4
q_tr = Inf
q_exit = Inf
solver, solver_options = ECOS, Dict("verbose"=>0)      

pars = PTR.Parameters(N, Nsub, iter_max, disc_method, wvc, wtr, ε_abs,
                      ε_rel, feas_tol, q_tr, q_exit, solver, solver_options);

Alternative convex solver options:

In [ ]:
# solver, options = Gurobi, Dict("OutputFlag"=>0)  # works
# solver, options = OSQP, Dict("verbose"=>0)       # works
# solver, options = Mosek, Dict("MSK_IPAR_LOG"=>0)
# solver, options = Ipopt, Dict()                  # doesn't work
# solver, options = COSMO, Dict()                  # doesn't work
# solver, options = SCS, Dict()                    # doesn't work

## Solving the Trajectory Problem

Solving the problem amounts to initializing the SCP solver with its parameters, and providing the problem definition `pbm` from the API.

In [ ]:
ptr_pbm = PTR.create(pars, pbm)
sol, history = PTR.solve(ptr_pbm);

## Trajectory plot

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()

_sol = history.subproblems[end].sol

tf = _sol.p[1]
xd = _sol.xd
ud = _sol.ud
vd = _sol.vd
p_sol = _sol.p
td = sol.td*tf

Nc = 100
tc = LinRange(0, tf, Nc)
xc = hcat([sample(sol.xc, t) for t in LinRange(0, 1, Nc)]...)

# ax.plot(td, xd[1, :], marker=".", linestyle="none", label="x")

# ax.plot(td, xd[2, :], marker=".", markersize=10, linestyle="none", label="y-dt", zorder=100)
# ax.plot(td, xd[2, :], linestyle="-", label="y-ct")

ax.plot(td, xd[2, :], marker=".", markersize=10, linestyle="none", label="y-dt", zorder=100)
ax.plot(tc, xc[2, :], linestyle="-", label="y-ct")

ax.legend()

ax.set_xlabel("time")
;

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()
ax.axis("equal")

ax.plot(xd[1, :], xd[2, :], marker=".", markersize=10, linestyle="none", label="pos dt", zorder=100)
ax.plot(xc[1, :], xc[2, :], linestyle="-", label="pos ct")

span_gs = max(100, 2*abs(x_0[1]))
x_cone = [-span_gs; 0; span_gs]
ax.plot(x_cone, tan(γ_gs).*abs.(x_cone), linestyle="--", color="red", label="gs")

ax.legend()

ax.set_xlabel("x pos")
ax.set_xlabel("y pos")
;

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()

ax.plot(td, xd[4, :], marker=".", markersize=10, linestyle="none", label="vy-dt", zorder=100)
ax.plot(tc, xc[4, :], linestyle="-", label="v-ct")

ax.legend()

ax.set_xlabel("time")
;

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()

ax.plot(td, xd[7, :], marker=".", markersize=10, linestyle="none", label="mass-dt", zorder=100)
ax.plot(tc, xc[7, :], linestyle="-", label="mass-ct")

ax.legend()

ax.set_xlabel("time")
;

In [ ]:
fig = plt.figure(figsize=(6, 8))
ax = fig.add_subplot(211)

ax.grid()

ax.plot(td, rad2deg.(xd[5, :]), marker=".", markersize=10, linestyle="none", label="y-dt", zorder=100)
ax.plot(tc, rad2deg.(xc[5, :]), linestyle="-", label="y-ct")

ax.legend()

ax.set_xlabel("time")
ax.set_ylabel("angle")

ax = fig.add_subplot(212)

ax.grid()

ax.plot(td, rad2deg.(xd[6, :]), marker=".", markersize=10, linestyle="none", label="y-dt", zorder=100)
ax.plot(tc, rad2deg.(xc[6, :]), linestyle="-", label="y-ct")

ax.legend()

ax.set_xlabel("time")
ax.set_ylabel("angular rate")
;

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()

uc = hcat([sample(sol.uc, t) for t in LinRange(0, 1, Nc)]...)

ax.plot(td, ud[1, :], marker=".", markersize=10, linestyle="none", label="T-dt", zorder=100)
ax.plot(tc, uc[1, :], linestyle="-", label="T-ct")

ax.axhline(y=T_min, color="red", linestyle="--")
ax.axhline(y=T_max, color="red", linestyle="--")

ax.legend()
tc
ax.set_xlabel("time")

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.grid()

ax.plot(td, rad2deg.(ud[2, :]), marker=".", markersize=10, linestyle="none", label="δ-dt", zorder=100)
ax.plot(tc, rad2deg.(uc[2, :]), linestyle="-", label="δ-ct")

ax.axhline(y=rad2deg(-δ_max), color="red", linestyle="--")
ax.axhline(y=rad2deg(δ_max), color="red", linestyle="--")

ax.legend()
tc
ax.set_xlabel("time")

In [ ]:
tf_values = [spbm.sol.p[1] for spbm in history.subproblems]
pushfirst!(tf_values, history.subproblems[1].ref.p[1])

fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.grid()
ax.plot(tf_values)
ax.set_xlabel("SCP iteration")
ax.set_ylabel("time dilation")
;

In [ ]:
fig = plt.figure(figsize=(12, 9))
for i = 1:7
    ax = fig.add_subplot(3, 3, i)
    ax.grid()
    ax.plot(td[1:end-1], vd[i, :], marker=".", markersize=5, linestyle="none", label="T-dt", zorder=100)
    ax.set_ylabel("vd$i")
    ax.set_xlabel("time")
end
plt.tight_layout()

In [ ]:
# function set_fonts()::Nothing
#     # Set the figure fonts.
#     fig_small_sz = 12
#     fig_med_sz = 15
#     fig_big_sz = 17
#     plt.rc("text", usetex=true)
#     plt.rc("font", size=fig_small_sz, family="serif")
#     plt.rc("axes", titlesize=fig_small_sz)
#     plt.rc("axes", labelsize=fig_med_sz)
#     plt.rc("xtick", labelsize=fig_small_sz)
#     plt.rc("ytick", labelsize=fig_small_sz)
#     plt.rc("legend", fontsize=fig_small_sz)
#     plt.rc("figure", titlesize=fig_big_sz)
#     plt.rc("figure", dpi=300) 
#     return nothing
# end
# ;

In [ ]:
# # Trajectory plot
# ctres, overlap = 1000, 3
# N = size(sol.xd, 2)
# xct = hcat([sample(sol.xc, t) for t in LinRange(0, 1, ctres)]...)
# vct = vcat([sample(sol.uc, t)[1] for t in LinRange(0, 1, ctres)]...)
# cmap = generate_colormap("inferno"; minval=minimum(vct), maxval=maximum(vct))

# # plot_options = Dict("xtick.labelsize"=>9,
# #                     "ytick.labelsize"=>9,
# #                     "axes.labelsize"=>11)
# # fig = create_figure((4, 4); options = plot_options)

# fig = plt.figure(figsize=(4,4), dpi=300)
# plt.clf()
# set_fonts()
# set_fonts()

# ax = setup_axis!(111, xlabel="\$x\$ [m]", ylabel="\$y\$ [m]",
#                  axis="equal", cbar=cmap, clabel="Velocity, \$v\$ [m/s]",
#                  cbar_aspect=40)
# ax.plot(sol.xd[1, :], sol.xd[2, :],
#         linestyle="none", marker="o", markerfacecolor=DarkBlue,
#         markeredgecolor="white", markeredgewidth=0.2, markersize=3,
#         zorder=20)
# line_segs = Vector{Matrix}(undef, 0)
# line_clrs = Vector{NTuple{4, Real}}(undef, 0)
# for k=1:ctres-overlap
#     push!(line_segs, xct[1:2, k:k+overlap]')
#     push!(line_clrs, cmap.to_rgba(vct[k]))
# end
# trajectory = PyPlot.matplotlib.collections.LineCollection(
#     line_segs, zorder=10, colors = line_clrs, linewidths=3,
#     capstyle="round")
# ax.add_collection(trajectory)
# Rect = PyPlot.matplotlib.patches.Rectangle
# car_length = 0.2
# for k=1:N
#     local xl, xw = [1;1;-1;-1;1]*car_length/2, [1;-1;-1;1;1]*car_width/2
#     local yl, yw = [1;1;-1;-1;1]*car_length/2, [-1;1;1;-1;-1]*car_width/2
#     local ang = sol.xd[3,k]
#     local xc = sol.xd[1,k].+xl.*sin(ang).+xw.*cos(ang)
#     local yc = sol.xd[2,k].+yl.*cos(ang).+yw.*sin(ang)
#     ax.fill(xc, yc,
#             linewidth=1,
#             edgecolor=DarkBlue,
#             facecolor=rgb2pyplot(parse(RGB, Red), a=0.5),
#             zorder=6)
# end
# ang = LinRange(0, 2*pi, 100)
# obs = ([cos.(ang)'; sin.(ang)']*r_0).+c_0
# ax.fill(obs[1, :], obs[2, :],
#         linewidth=1,
#         edgecolor=Blue,
#         facecolor=rgb2pyplot(parse(RGB, Green), a=0.5),
#         zorder=5)

# fig.savefig("../media/outputs/dubin_trajectory.png", dpi="figure", bbox_inches=nothing)

# plt.close()

## Replace figure

<!-- <div style="text-align: center">
    <img src="../media/outputs/dubin_trajectory.png"
         alt="Dubin's car overview"
         style="background-color: white; width: 500px; display: block; margin-left: auto; margin-right: auto;"/>
    <br />
    <b>Figure.</b> Computed Dubin's car trajectory for minimum-input usage.
</div> -->